# Piezo Crystal Voltage Responce

In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mp
import matplotlib.pyplot as plt 
import uncertainties as unc
import scipy as sci

from uncertainties import ufloat
from scipy.optimize import curve_fit

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Import data

In [2]:
# Function Generator Voltage
FV = pd.read_csv('Data/F0002CH1.CSV', header=None, index_col=False, usecols=[3, 4])

# Piezo Voltage
PV = pd.read_csv('Data/F0002CH4.CSV', header=None, index_col=False, usecols=[3, 4])

# Photodetector Voltage 
DV = pd.read_csv('Data/F0002CH2.CSV', header=None, index_col=False, usecols=[3, 4])

In [3]:
# FV.rename(columns={'3':'Time (s)', '4':'Voltage (s)'}, inplace=True)
FV.columns = ['Time (s)', 'Voltage (V)']
PV.columns = ['Time (s)', 'Voltage (mV)']
DV.columns = ['Time (s)', 'Voltage (V)']

## Plot the Data

In [4]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

FV.plot(x = 'Time (s)', y = 'Voltage (V)' , ax = axes[0], legend=False,
        title = 'Voltage output of function generator',
        xlabel = 's',
        ylabel = 'V')

PV.plot(x = 'Time (s)', y = 'Voltage (mV)', ax = axes[1], legend=False,
        title = 'Voltage applied to piezo crystal',
        xlabel = 's',
        ylabel = 'mV')

DV.plot(x = 'Time (s)', y = 'Voltage (V)', ax = axes[2], legend=False,
        title = 'Voltage output of photodetector',
        xlabel = 's',
        ylabel = 'V')

axes[0].grid()
axes[1].grid()
axes[2].grid()
plt.tight_layout()
plt.show()

<Figure size 864x216 with 3 Axes>

## Clean up data

In [5]:
# Normalize photodetector intensity
DV['Intensity'] = (DV['Voltage (V)'] - np.min(DV['Voltage (V)'])) / (np.max(DV['Voltage (V)']) - np.min(DV['Voltage (V)']))

In [6]:
# Smoothing outputs 
# =================

# Hanning smoothing 
def smooth(x, window_len=11, window='hanning'):
    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y[int((window_len-1)/2):-int((window_len-1)/2)]

HanIntFV, HanIntPV, HanIntDV, HanIntDVIntn = 21, 21, 21, 21 

FVHan = smooth(FV['Voltage (V)'], window_len=HanIntFV)
PVHan = smooth(PV['Voltage (mV)'], window_len=HanIntPV)
DVHan = smooth(DV['Voltage (V)'], window_len=HanIntDV)
DVHanIntn = smooth(DV['Intensity'], window_len=HanIntDVIntn)

## Curve fit the voltage output of photodetector

In [7]:
# Function
def DVFunc(x, a, b, c, d, e, a2, b2, c2):
    f = a2*np.sin(np.pi*b2*x + c2*np.pi)**2
    F = a*np.sin(np.pi*(b + f)*x + c*np.pi)**2 + d + e*x
    return F

def LinFunc(x, a, b):
    return a*x + b

In [8]:
# Introduce Cuts
# ==============
# Find positions of min and max values of PV
min_pos, max_pos = np.argmin(PV['Voltage (mV)']), np.argmax(PV['Voltage (mV)'])

Cut0Pos = min(min_pos, max_pos)
Cut1Pos = max(min_pos, max_pos)
Cut1 = PV['Voltage (mV)'][Cut1Pos:]

min_pos2, max_pos2 = np.argmin(Cut1)+Cut1Pos, np.argmax(Cut1)+Cut1Pos

Cut2Pos = max(min_pos2, max_pos2)

# Zero time
DV['Time Zeroed (s)'] = DV['Time (s)'] - DV['Time (s)'][Cut0Pos]
PV['Time Zeroed (s)'] = PV['Time (s)'] - PV['Time (s)'][Cut0Pos]

# Constrain DV and PV elements to those between min and max of PV
DVTime, DVVolt = DV['Time Zeroed (s)'][Cut0Pos:Cut1Pos], DVHan[Cut0Pos:Cut1Pos]
PVTime, PVVolt = PV['Time Zeroed (s)'][Cut0Pos:Cut1Pos], PVHan[Cut0Pos:Cut1Pos]

DVTime2, DVVolt2 = DV['Time Zeroed (s)'][Cut1Pos:Cut2Pos], DVHan[Cut1Pos:Cut2Pos]
PVTime2, PVVolt2 = PV['Time Zeroed (s)'][Cut1Pos:Cut2Pos], PVHan[Cut1Pos:Cut2Pos]

### Curve fit raw data

In [32]:
# Up to cut 1 
DVpopt, DVpcov = curve_fit(DVFunc, DVTime, DVVolt, 
#                            p0 = [0.5, 400, 1, -0.5, 0, 0, 0, 0], 
                       bounds = ([ 0.25, 300, -2, -1, -2, -np.inf,    0,       0], 
                                 [ 0.35, 800,  2,  0,  2,  np.inf,  100,  np.inf])
                          )

PVpopt, PVpcov = curve_fit(LinFunc, PVTime, PVVolt)

a1, b1, c1, d1, e1, a12, b12, c12 = DVpopt

In [33]:
# After cut 1
DVpopt2, DVpcov2 = curve_fit(DVFunc, DVTime2, DVVolt2, 
#                            p0 = [0.5, 1020, 1, -0.5, 0, 0, 0, 0], 
                       bounds = ([ 0.25,  200, -2, -1, -2, -np.inf,   0,       0], 
                                 [ 0.35,  800,  2,  0,  2,  np.inf, 100,  np.inf])
                          )

PVpopt2, PVpcov2 = curve_fit(LinFunc, PVTime2, PVVolt2)

a2, b2, c2, d2, e2, a22, b22, c22 = DVpopt2

In [34]:
# # Normalized Photodetector intensity
# DVIntn = DVHanIntn[Cut0Pos:Cut1Pos]/np.mean([a1, a2])
# DVIntn2 = DVHanIntn[Cut1Pos:Cut2Pos]/np.mean([a1, a2])

# # Convert time to space
# lam = 632 # (nm)

# DV['Displacement (nm)'] = (DV['Time (s)'] - DV['Time (s)'][Cut0Pos])*lam*np.mean([b1, b2])/2 
# DVDisp = DV['Displacement (nm)'][Cut0Pos:Cut1Pos]
# DVDisp2 = DV['Displacement (nm)'][Cut1Pos:Cut2Pos]

# # Curve Fit
# DVIpopt, DVIpcov = curve_fit(DVFunc, DVDisp, DVIntn, # method='dogbox',
#                              p0 = [0.42, 0.0026, 20, 0.5, 0,  0, 0, 0],
#                              bounds = ([ 0.4, -0.0025, -np.inf,  0.47, -10e-5,  -np.inf, -np.inf, -np.inf], 
#                                        [ 0.5,  0.0027,  np.inf,  0.53,  10e-5,   np.inf,  np.inf,  np.inf]) 
#                             )
# aI1, bI1, cI1, dI1, eI1, aI12, bI12, cI12 = DVIpopt

# DVIpopt2, DVIpcov2 = curve_fit(DVFunc, DVDisp2, DVIntn2, # method='dogbox',
#                              p0 = [0.42, 0.0026, 20, 0.5, 0,  0, 0, 0],
#                              bounds = ([ 0.4, -0.0025, -np.inf,  0.47, -10e-5,  -np.inf, -np.inf, -np.inf], 
#                                        [ 0.5,  0.0027,  np.inf,  0.53,  10e-5,   np.inf,  np.inf,  np.inf]) 
#                             )

# aI2, bI2, cI2, dI2, eI2, aI22, bI22, cI22 = DVIpopt2

## Plotting out the data

In [35]:
# Plot it out 
fig, ax = plt.subplots(2, 1, figsize = (8, 10))

# Voltage applied to piezo  per time
PV.plot(x = 'Time Zeroed (s)', y = 'Voltage (mV)', ax = ax[0], legend=False,
        title = 'Voltage applied to piezo crystal',
        xlabel = 's',
        ylabel = 'mV')

ax[0].plot(PVTime, LinFunc(PVTime, *PVpopt), c='orange', zorder=3,
           label = r'Fit: $V = ({0:.4g}) t  {1:+.4g}$'.format(PVpopt[0], PVpopt[1]))

ax[0].plot(PVTime2, LinFunc(PVTime2, *PVpopt2), c='red', zorder=3,
           label = r'Fit: $V = ({0:.4g}) t  {1:+.4g}$'.format(PVpopt2[0], PVpopt2[1]))

ax[0].plot(PV['Time Zeroed (s)'], PVHan, c='black',
           label = 'Hanning over {0} samples'.format(HanIntPV))

ax[0].legend()
ax[0].grid()

# Voltage output of photodetector per time
DV.plot(x = 'Time Zeroed (s)', y = 'Voltage (V)', legend=False, ax = ax[1],
        title = 'Voltage output of photodetector',
        xlabel = 's',
        ylabel = 'V',
        label = 'Data')

ax[1].plot(DVTime, DVFunc(DVTime, *DVpopt), c='orange', zorder=3,
          label = 
               r'Fit: $V = {0:.4g} \sin^2(\nu \pi t {2:+.4g} \pi) {3:+.4g} {4:+.4g}t$' '\n' 
               r'Where $\nu= {1:.4g} {5:+.4g} \sin^2 ({6:.4g} \pi t {7:+.4g}\pi)$'
               .format(a1, b1, c1, d1, e1, a12, b12, c12))

ax[1].plot(DVTime2, DVFunc(DVTime2, *DVpopt2), c='red', zorder=3,
          label = 
               r'Fit: $V = {0:.4g} \sin^2(\nu \pi t {2:+.4g} \pi) {3:+.4g} {4:+.4g}t$' '\n'
               r'Where $\nu= {1:.4g} {5:+.4g} \sin^2 ({6:.4g} \pi t {7:+.4g}\pi)$'
               .format(a2, b2, c2, d2, e2, a22, b22, c22))

ax[1].plot(DV['Time Zeroed (s)'], DVHan, c='black', zorder=2,
           label = 'Hanning over {0} samples'.format(HanIntDV))

ax[1].set_ylim(np.min(DV['Voltage (V)'])*1.4)
ax[1].legend(loc='lower left')
ax[1].grid()

# Normalized intensity of photodetector with time
# DV.plot(x = 'Displacement (nm)', y = 'Intensity', legend=False, ax=ax[2],
#         title = 'Normalized output of photodetector',
#         xlabel = 'nm',
#         ylabel = 'V',
#         label = 'Data')

# ax[2].plot(DVDisp, DVFunc(DVDisp, *DVIpopt), c='orange', zorder=3,
#           label = 
#                r'Fit: $y = ({0:.2g}) \sin(\nu \pi x {2:+.2g} \pi) {3:+.2g} {4:+.2g}x$' '\n' 
#                r'Where $\nu= ({1:.3g}) {5:+.3g} \sin^2 ({6:.3g} \pi x {7:+.3g}\pi)$'
#                .format(aI1, bI1, cI1, dI1, eI1, aI12, bI12, cI12))

# ax[2].plot(DVDisp2, DVFunc(DVDisp2, *DVIpopt2), c='red', zorder=3,
#           label = 
#                r'Fit: $y = ({0:.2g}) \sin(\nu \pi x {2:+.2g} \pi) {3:+.2g} {4:+.2g}x$' '\n' 
#                r'Where $\nu= ({1:.3g}) {5:+.3g} \sin^2 ({6:.3g} \pi x {7:+.3g}\pi)$'
#                .format(aI2, bI2, cI2, dI2, eI2, aI22, bI22, cI22))

# ax[2].plot(DV['Displacement (nm)'], DVHanIntn, c='black', zorder=2,
#            label = 'Hanning over {0} samples'.format(HanIntDVIntn))

# ax[2].set_ylim(np.min(DV['Intensity']) - 0.55)
# ax[2].legend(loc='lower left')
# ax[2].grid()

# plt.tight_layout()
# plt.show()

<Figure size 576x720 with 2 Axes>

## Displacement of Piezo vs Voltage

In [13]:
# Plot piezo displacement vs Voltage
IntNormDat = DVIntn
IntNormFit = DVFunc(DVTime, *DVIpopt)
# PiezoDisp = (DVDisp-np.min(DVDisp))/2

def IntToDispFunc(x, a=aI, b=bI, c=cI, d=dI, lam=632, f=2):
    A = (x-d)/a
    Num = np.arcsin(A) - c*np.pi
    Den = b*np.pi 
    F = Num/Den
    return F*lam/f

# Disppopt, Disppcov = curve_fit(LinFunc, PiezoDisp, PVVolt)

# Plot it out
#============

# Plot Data
plt.plot(IntToDispFunc(IntNormDat),
#          IntToDispFunc(IntNormDat)
#         np.unwrap(IntToDispFunc(IntNormDat))
         PVVolt)

# Plot Fit
# plt.plot(PiezoDisp, LinFunc(PiezoDisp, *Disppopt),
#          label = r'y = {0:.3g}x + {1:.3g}'.format(*Disppopt))

plt.xlabel('Displacement (nm)')
plt.ylabel('Voltage (V)')
plt.title('Piezo displacement per applied voltage')
plt.legend()
plt.grid()
plt.show()

NameError: name 'DVIntn' is not defined

In [ ]:
# DVIntn, np.unwrap(IntToDispFunc(IntNormDat))

### Graphing the voltage output of the photodetector against voltage applied to 

In [ ]:

plt.plot(PV['Voltage (mV)'], DV['Intensity'])

plt.plot(PVHan, DVHanIntn, c='black')


## Figuring out the displacement of piezo crystal per volt

Wavelength of the laser: $\lambda = 632$ nm

Frequency of the photodetector: $\nu_D = 3969.668$ Hz or $\lambda / s$

This means that the piezo is moving at $v_P = \nu_D / 2$ $\lambda/s$

For that period, the voltage applied to the piezo is increasing at $1.7 \times 10^4$ mV/s = $17$ V/s



In [ ]:
T1 = np.arange(-5*np.pi, 5*np.pi, 0.001)
T2 = np.unwrap(T1)
print(T1, T2)

In [ ]:
plt.plot(T1, T2)